In [1]:
import glob, ipyleaflet, IPython.display, json, matplotlib.cm, os, pyproj, shapely.geometry, scipy.spatial, sys, pdal
import ipyvolume.pylab as p3
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from scipy.spatial import ConvexHull



from local_max import local_max

In [2]:
# data source:
# http://gis.arso.gov.si/evode/profile.aspx?id=atlas_voda_Lidar@Arso&culture=en-US

data_dir = r'data'
file_name = r'Trees_flat_field_D48GK.las' 
file_path = os.path.abspath(os.path.join(data_dir, file_name))

In [3]:
pipeline = {
    "pipeline":[file_path, 
                {"type":"filters.hag_nn"}, 
                {"type":"filters.eigenvalues", "knn":16}, 
                {"type":"filters.normal", "knn":16}] 
}

#processing pipeline  



In [4]:
pipeline = pdal.Pipeline(json.dumps(pipeline))
pipeline.validate()
%time n_points = pipeline.execute()

arr = pipeline.arrays[0] 

description = arr.dtype.descr
cols = [col for col, __ in description]

df = pd.DataFrame({col: arr[col] for col in cols}) #create a pandas dataframe from array


#localise coordinates to easily plot them
df['X_0'] = df['X'] - df['X'].min()
df['Y_0'] = df['Y'] - df['Y'].min()
df['Z_0'] = df['Z'] - df['Z'].min()




Wall time: 25.2 s


In [5]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,...,Eigenvalue0,Eigenvalue1,Eigenvalue2,NormalX,NormalY,NormalZ,Curvature,X_0,Y_0,Z_0
0,447451.03125,115474.640625,376.089996,16,1,2,1,0,5,-22.0,...,0.020981,0.038285,0.064077,-0.226533,-0.090666,0.969774,0.170105,0.34375,0.484375,15.589996
1,447451.87500,115474.562500,380.950012,8,1,1,1,0,5,-22.0,...,0.005230,0.025840,0.030152,0.040012,-0.556094,0.830156,0.085424,1.18750,0.406250,20.450012
2,447451.40625,115474.757813,376.290009,12,1,3,1,0,5,-22.0,...,0.009641,0.036276,0.077016,0.530990,0.200247,0.823378,0.078427,0.71875,0.601563,15.790009
3,447451.18750,115476.351563,385.239990,79,1,4,1,0,5,-22.0,...,0.006375,0.031941,0.044325,0.022861,0.270722,0.962386,0.077137,0.50000,2.195313,24.739990
4,447451.71875,115476.007813,383.700012,57,2,4,1,0,5,-22.0,...,0.003562,0.031965,0.106745,-0.234453,0.199693,0.951396,0.025038,1.03125,1.851563,23.200012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795922,447577.46875,115588.343750,388.279999,185,1,2,1,0,5,14.0,...,0.030218,0.089484,0.405468,0.420646,-0.533102,0.734070,0.057539,126.78125,114.187500,27.779999
795923,447577.37500,115588.812500,386.540009,30,2,2,1,0,5,14.0,...,0.122189,0.143618,0.448972,-0.992957,-0.108317,0.047995,0.170946,126.68750,114.656250,26.040009
795924,447577.59375,115588.648438,388.579987,74,1,4,1,0,5,14.0,...,0.034981,0.098673,0.319598,0.719064,-0.289574,0.631739,0.077178,126.90625,114.492188,28.079987
795925,447577.53125,115588.812500,387.989990,49,2,4,1,0,5,14.0,...,0.039391,0.177403,0.395562,0.993115,-0.060844,0.100105,0.064326,126.84375,114.656250,27.489990


In [8]:
fig = p3.figure(width=1000)

fig.xlabel='Y'
fig.ylabel='X'
fig.zlabel='Z'
all_points_p3 = p3.scatter(df['Y_0'], df['X_0'], df['Z_0'], color='red', size=.2) #if you want to plot all points

p3.squarelim()


In [9]:
ground = df[df['Classification'] == 2].reset_index() # The data is classified so we just use the classification
tree_points = df[(df['Classification'] == 5) & 
                    (df['HeightAboveGround'] >= 1) & 
                    (df['Eigenvalue0'] > .05) & 
                    (df['NumberOfReturns'] - df['ReturnNumber'] >= 1)].reset_index() #Separating trees 

In [11]:
tree_points_p3 = p3.scatter(tree_points['Y_0'], tree_points['X_0'], tree_points['Z_0'], color='darkgreen', size=.2)

ground_delaunay = scipy.spatial.Delaunay(ground[['X_0','Y_0']]) # create 3d mesh from ground points
ground_p3 = p3.plot_trisurf(ground['Y_0'], ground['X_0'], ground['Z_0'], ground_delaunay.simplices, color='grey')
fig.meshes.append(ground_p3)

all_points_p3.visible = False

In [ ]:
p3.show()

In [12]:
tree_tops = local_max(tree_points, radius=3, density_threshold=15).reset_index()

# find local maximums which represent treetops - this can be used to count trees
tree_local_max_p3 = p3.scatter(tree_tops['Y_0'], tree_tops['X_0'], tree_tops['Z_0'], color='red', size=1, marker='sphere')
tree_local_max_p3.color = matplotlib.cm.tab10(np.arange(len(tree_tops['Z_0']))%10)[:,0:3]

kdtree = scipy.spatial.kdtree.KDTree(tree_tops[['Y_0','X_0','Z_0']])  # separate trees and set tree indexes
dist, idx = kdtree.query(tree_points[['Y_0','X_0','Z_0']].values)
tree_points_p3.color=matplotlib.cm.tab10(idx%10)[:,0:3]  # color separate trees

tree_points['idx'] = idx


In [ ]:
tree_points

In [13]:
medians = tree_points.groupby('idx')[['Y_0','X_0','Z_0']].median()

# aproximate tree canopy volume and plot them as spheres
for axis in ['Y_0','X_0','Z_0']:
    tree_points['d'+axis[0]] = tree_points[axis] - tree_points['idx'].map(medians[axis])
tree_points['radius'] = np.linalg.norm(tree_points[['dY', 'dX', 'dZ']].values, axis=1)
radius = pd.DataFrame([tree_points.groupby('idx')['radius'].quantile(.5), tree_tops['HeightAboveGround'].values*.4]).min()



In [14]:
#move around existing treetop spheres
tree_local_max_p3.x = medians['Y_0']
tree_local_max_p3.y = medians['X_0']
tree_local_max_p3.z = medians['Z_0']

tree_local_max_p3.size = radius

In [ ]:
# if you want to be serious, you can cycle through all of the clusters and create convex hulls for each
# this can take a while

for id in tree_points['idx'].values:
    tree_id = tree_points[tree_points['idx']==id]
    hull = ConvexHull([[x, y, z] for x, y, z in zip(tree_id['Y_0'].values, tree_id['X_0'].values, tree_id['Z_0'].values)])
    hull_p3 = p3.plot_trisurf(tree_id['Y_0'], tree_id['X_0'], tree_id['Z_0'], hull.simplices, color='green')

In [10]:
p3.show()